In [11]:
import cv2

def find_initial_piano_frame(video_path, max_frames=100):
    cap = cv2.VideoCapture(video_path)
    for i in range(max_frames):
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Simple threshold to detect white/black keys pattern
        _, thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
        white_ratio = cv2.countNonZero(thresh) / (thresh.shape[0] * thresh.shape[1])
        if 0.3 < white_ratio < 0.7:  # Heuristik
            cap.release()
            return i, frame
    cap.release()
    raise Exception("Tidak ditemukan frame awal dengan pola tuts.")


In [12]:
import cv2
import numpy as np

def detect_keys(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    vertical_sum = np.sum(edges, axis=0)
    threshold = np.max(vertical_sum) * 0.5
    black_key_positions = [i for i, v in enumerate(vertical_sum) if v > threshold]

    # Grouping (smoothing noise)
    grouped = []
    current = []
    for i in black_key_positions:
        if not current or i - current[-1] < 5:
            current.append(i)
        else:
            grouped.append(current)
            current = [i]
    if current:
        grouped.append(current)

    black_keys = [int(np.mean(g)) for g in grouped]

    # Estimasi white keys: ambil jarak antar black keys, sisipkan putih di antaranya
    white_keys = []
    for i in range(len(black_keys)-1):
        mid = (black_keys[i] + black_keys[i+1]) // 2
        white_keys.append(mid)
    return white_keys, black_keys


In [13]:
import cv2

def track_note_events(video_path, white_keys, black_keys, start_frame):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    events_by_color = {}

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    prev_frame = None
    frame_idx = start_frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        roi = frame[frame.shape[0]//2:, :]  # ambil bagian bawah video
        for i, x in enumerate(white_keys):
            color = tuple(roi[10, x])  # ambil warna di titik x
            color_str = str(color)
            note = f"key_{i}"
            if color_str not in events_by_color:
                events_by_color[color_str] = []
            events_by_color[color_str].append({
                "note": note,
                "start": frame_idx / fps,
                "end": (frame_idx + 1) / fps
            })
        frame_idx += 1
    cap.release()
    return events_by_color


In [14]:
import librosa

def estimate_tempo(audio_path):
    y, sr = librosa.load(audio_path)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    return round(tempo)


In [15]:
from music21 import stream, note, tempo, metadata

def write_sheet(events_by_color, bpm):
    s = stream.Score()
    s.insert(0, tempo.MetronomeMark(number=bpm))
    s.insert(0, metadata.Metadata(title="Synthesia Analysis"))

    for color, notes in events_by_color.items():
        p = stream.Part()
        for n in notes:
            dur = n["end"] - n["start"]
            m21_note = note.Note(n["note"].replace("key_", "C"), quarterLength=max(0.25, dur * bpm / 60))
            p.append(m21_note)
        s.append(p)

    s.write("musicxml", fp="output/synthesia_output.musicxml")
    print("🟢 File disimpan ke output/synthesia_output.musicxml")


In [16]:


video_path = "../../Data/video/Yakuza_6_-_Destiny_Piano_Cover.mp4"
audio_path = None  # opsional




print("🔍 Mencari frame awal...")
start_frame_num, initial_frame = find_initial_piano_frame(video_path)

print("🎹 Mendeteksi tuts piano...")
white_keys, black_keys = detect_keys(initial_frame)

print("🎨 Melacak warna Synthesia dan nada...")
events_by_color = track_note_events(video_path, white_keys, black_keys, start_frame_num)

print("⏱️ Mendeteksi tempo lagu...")
tempo = estimate_tempo(audio_path or video_path)

print("📝 Menulis lembaran musik...")
write_sheet(events_by_color, tempo)

print("✅ Proses selesai. Lembaran musik telah dibuat!")

🔍 Mencari frame awal...


Exception: Tidak ditemukan frame awal dengan pola tuts.